# Create phenology datasets and export them to google drive

In [35]:
# load required modules
import ee
ee.Initialize()
import pandas as pd
import numpy as np

In [ ]:
# load leaflet interactive map
import geemap
Map = geemap.Map()

### MCD12Q2 v006 phenology product

In [ ]:
# function for extracting quality bits
def getQABits(image, start, end, mascara):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start,end+1):
        pattern += 2**i
    # Return a single band image of the extracted QA bits, giving the     band a new name.
    return image.select([0], [mascara]).bitwiseAnd(pattern).rightShift(start)
    
# function for filtering image based on quality bits
def mask_pixelsall(bstart,bend):
    def maskPixels(image0):
        tmp = image0.select('QA_Detailed_1')
        quality = getQABits(tmp, bstart, bend, 'QA_Detailed_1')
        # Create a mask that filters out undesired areas
        mask = quality.eq(0).Or(quality.eq(1)).Or(quality.eq(2))
        return image0.updateMask(mask) 
    return(maskPixels)

# function for masking areas where there have not land use changes. This layer was created using the ESA cci map
def mask_image(image):
    image = image.updateMask(MakMarco.eq(1))
    return image

# convert feature collection into feature collection with no geometry (easier to save)
def convert(feature):
    res = ee.Feature(None,feature.toDictionary())
    return(res)

In [ ]:
# ---- load datasets
# load MODIS phenology product
modis_phenoprod = ee.ImageCollection('MODIS/006/MCD12Q2')
# load mask of unchanged forest pixels
MakMarco = ee.Image("users/marcogirardello/phenoutils/mask_unchanged_500m")
# 5 km x 5 km grid
mask_5km = ee.Image('users/marcogirardello/phenoutils/pheno_grid').int()
# broad areas
export_grid =ee.FeatureCollection('users/marcogirardello/phenoutils/grid_export_phenology')

In [36]:
# polygon numbers for broad areas
polygons = list(range(1, 42+1))

# list of years
years = list(range(2001, 2016+1))

for year in years:
    print(year)
    # filter for year of interest
    pheno_tmp = modis_phenoprod.filter(ee.Filter.date(str(year)+'-01-01', str(year+1)+'-01-01'))
    # ------- Greenup
    GUP = pheno_tmp.select(['Greenup_1','QA_Detailed_1'])
    # only take pixels above a certain threshold level
    GUP1 = GUP.map(mask_pixelsall(bstart=0,bend=1)).select('Greenup_1').first()
    # filter using mask where pixels have been stable
    GUP2 = GUP1.updateMask(MakMarco.eq(1))
    # reproject latlon
    GUP3 = GUP2.reproject(crs='EPSG:4326',scale= 463.3127165275)
    # export as CSV file
    for polygon in polygons:
        #print(polygon)
        # subset one broad area
        onesquare = export_grid.filterMetadata('polyID','equals',polygon)
        # convert 5km grid into vectors (pixels become polygons!)
        vectors = mask_5km.reduceToVectors(crs = mask_5km.projection(),geometry = onesquare,scale =100,
                                           geometryType = 'polygon',eightConnected = False, labelProperty ='zone',
                                           reducer= ee.Reducer.countEvery(),maxPixels= 1e13)
        # calculate statistics of interest (standard deviation)
        stats = GUP3.reduceRegions(collection = vectors,reducer = ee.Reducer.stdDev(),
                                          scale = 463.3127165275)
        # convert to dictionary (set geometry to null!)
        stats1 = stats.map(convert)
        # filename 
        filename = 'GUP_sd_'+str(year)+'_'+str(polygon)
        #Export the FeatureCollection.
        task= ee.batch.Export.table.toDrive(collection = stats1,description =filename,folder="phenology_csv",
                                        fileFormat='CSV')
        task.start()
        
    # ------- Peak
    # subset Peak for given year
    Peak = pheno_tmp.select(['Peak_1','QA_Detailed_1'])
    # only take pixels above a certain threshold level
    Peak1 = Peak.map(mask_pixelsall(bstart=6,bend=7)).select('Peak_1').first()
    # filter using mask where pixels have been stable
    Peak2 = Peak1.updateMask(MakMarco.eq(1))
    # reproject latlon
    Peak3 = Peak2.reproject(crs='EPSG:4326',scale= 463.3127165275)
    # export as CSV file
    for polygon in polygons:
        #print(polygon)
        # subset one broad area
        onesquare = export_grid.filterMetadata('polyID','equals',polygon)
        # convert 5km grid into vectors (pixels become polygons!)
        vectors = mask_5km.reduceToVectors(crs = mask_5km.projection(),geometry = onesquare,scale =100,
                                           geometryType = 'polygon',eightConnected = False, labelProperty ='zone',
                                           reducer= ee.Reducer.countEvery(),maxPixels= 1e13)
        # calculate statistics of interest (standard deviation)
        stats = Peak3.reduceRegions(collection = vectors,reducer = ee.Reducer.stdDev(),
                                          scale = 463.3127165275)
        # convert to dictionary (set geometry to null!)
        stats1 = stats.map(convert)
        # filename 
        filename = 'Peak_sd_'+str(year)+'_'+str(polygon)
        #Export the FeatureCollection.
        task= ee.batch.Export.table.toDrive(collection = stats1,description =filename,folder="phenology_csv",
                                        fileFormat='CSV')
        task.start()
        
    # ------- Dormancy
    Dormancy = pheno_tmp.select(['Dormancy_1','QA_Detailed_1'])
    # only take pixels above a certain threshold level
    Dormancy1 = Dormancy.map(mask_pixelsall(bstart=12,bend=13)).select('Dormancy_1').first()
    # filter using mask where pixels have been stable
    Dormancy2 = Dormancy1.updateMask(MakMarco.eq(1))
    # calculate standard deviation
    Dormancy3 = Dormancy2.reproject(crs='EPSG:4326',scale= 463.3127165275)
    # export as CSV file
    for polygon in polygons:
        #print(polygon)
        # subset one broad area
        onesquare = export_grid.filterMetadata('polyID','equals',polygon)
        # convert 5km grid into vectors (pixels become polygons!)
        vectors = mask_5km.reduceToVectors(crs = mask_5km.projection(),geometry = onesquare,scale =100,
                                           geometryType = 'polygon',eightConnected = False, labelProperty ='zone',
                                           reducer= ee.Reducer.countEvery(),maxPixels= 1e13)
        # calculate statistics of interest (standard deviation)
        stats = Dormancy3.reduceRegions(collection = vectors,reducer = ee.Reducer.stdDev(),
                                          scale = 463.3127165275)
        # convert to dictionary (set geometry to null!)
        stats1 = stats.map(convert)
        # filename 
        filename = 'Dormancy_sd_'+str(year)+'_'+str(polygon)
        #Export the FeatureCollection.
        task= ee.batch.Export.table.toDrive(collection = stats1,description =filename,folder="phenology_csv",
                                        fileFormat='CSV')
        task.start()
    # ------- Season length
    season_length = Dormancy2.subtract(GUP2)
    season_length1 = season_length.reproject(crs='EPSG:4326',scale= 463.3127165275)
    # export as CSV file
    for polygon in polygons:
        #print(polygon)
        # subset one broad area
        onesquare = export_grid.filterMetadata('polyID','equals',polygon)
        # convert 5km grid into vectors (pixels become polygons!)
        vectors = mask_5km.reduceToVectors(crs = mask_5km.projection(),geometry = onesquare,scale =100,
                                           geometryType = 'polygon',eightConnected = False, labelProperty ='zone',
                                           reducer= ee.Reducer.countEvery(),maxPixels= 1e13)
        # calculate statistics of interest (standard deviation)
        stats = season_length1.reduceRegions(collection = vectors,reducer = ee.Reducer.stdDev(),
                                          scale = 463.3127165275)
        # convert to dictionary (set geometry to null!)
        stats1 = stats.map(convert)
        # filename 
        filename = 'SL_sd_'+str(year)+'_'+str(polygon)
        #Export the FeatureCollection.
        task= ee.batch.Export.table.toDrive(collection = stats1,description =filename,folder="phenology_csv",
                                        fileFormat='CSV')
        task.start()

2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
